In [1]:
import pandas as pd     
from SmilesPE.pretokenizer import atomwise_tokenizer
import pdb      #pdb.set_trace()

In [2]:
le_s1 = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_1.txt', usecols=['CMP_CHEMBL_ID'])
le_s2 = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_2.txt', usecols=['CMP_CHEMBL_ID'])
le_s3 = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_3.txt', usecols=['CMP_CHEMBL_ID'])
le_s4 = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_4.txt', usecols=['CMP_CHEMBL_ID'])
le_s5 = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_5.txt', usecols=['CMP_CHEMBL_ID'])

In [3]:
lense = le_s1.append(le_s2)
lense = lense.append(le_s3)
lense = lense.append(le_s4)
lense = lense.append(le_s5)

In [4]:
chembl = pd.read_table('../datasets/ChEMBL/chembl_27_chemreps.txt', usecols=['chembl_id', 'canonical_smiles'])

In [5]:
lense = pd.merge(lense, chembl,
                how='left',
                left_on='CMP_CHEMBL_ID', right_on='chembl_id')

In [6]:
lense = lense[['chembl_id', 'canonical_smiles']]

In [7]:
lense.dropna(how='any', inplace=True)
lense.reset_index(drop=True)

,chembl_id,canonical_smiles
0,CHEMBL3237949,CCOC(=O)C1=C(N)N(c2cccnc2)C2=C(C(=O)CC(C)(C)C2...
1,CHEMBL2381895,CC(C)(C)c1ccccc1Oc1ncccc1NC(=O)Nc1ccc(N2CCCCC2...
2,CHEMBL2380718,CCC(NC(=O)[C@@H]1C[C@@H](S(=O)(=O)c2ccccc2)CN1...
3,CHEMBL2435858,Cc1nc(Oc2ccc(C(N)=O)cc2)ccc1CN1CCC(N2C(=O)N(C3...
4,CHEMBL2413735,Cc1ccc2[nH]c(-c3ccc(OCCN4CCCC4)cc3)nc2c1
...,...,...
992,CHEMBL2032165,CCCC1c2ccsc2CCN1C(=O)Nc1ccccc1
993,CHEMBL2153490,N#CCCN(CCn1cnc2c(=O)[nH]cnc21)CCP(=O)(O)O
994,CHEMBL1766798,CC(CCO)S(=O)(=O)c1ccc(-c2cnc(N)c(C(=O)Nc3ccccc...
995,CHEMBL1198756,N#C[C@@H]1CSCN1C(=O)CN(N)C1CCN(CC(=O)NC2CCCCC2...


In [8]:
lense['tokens'] = ''
for row in lense.index:
    lense.at[row, 'tokens'] = atomwise_tokenizer(lense.at[row, 'canonical_smiles'])

In [9]:
lense.head()

,chembl_id,canonical_smiles,tokens
0,CHEMBL3237949,CCOC(=O)C1=C(N)N(c2cccnc2)C2=C(C(=O)CC(C)(C)C2...,"[C, C, O, C, (, =, O, ), C, 1, =, C, (, N, ), ..."
1,CHEMBL2381895,CC(C)(C)c1ccccc1Oc1ncccc1NC(=O)Nc1ccc(N2CCCCC2...,"[C, C, (, C, ), (, C, ), c, 1, c, c, c, c, c, ..."
2,CHEMBL2380718,CCC(NC(=O)[C@@H]1C[C@@H](S(=O)(=O)c2ccccc2)CN1...,"[C, C, C, (, N, C, (, =, O, ), [C@@H], 1, C, [..."
3,CHEMBL2435858,Cc1nc(Oc2ccc(C(N)=O)cc2)ccc1CN1CCC(N2C(=O)N(C3...,"[C, c, 1, n, c, (, O, c, 2, c, c, c, (, C, (, ..."
4,CHEMBL2413735,Cc1ccc2[nH]c(-c3ccc(OCCN4CCCC4)cc3)nc2c1,"[C, c, 1, c, c, c, 2, [nH], c, (, -, c, 3, c, ..."


In [10]:
type(lense.tokens)

pandas.core.series.Series

In [11]:
alphabet_lense = set()

for i in lense.tokens:
    alphabet_lense.update(i)

In [12]:
alphabet_lense

{'#',
 '(',
 ')',
 '-',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '=',
 'Br',
 'C',
 'Cl',
 'F',
 'I',
 'N',
 'O',
 'P',
 'S',
 '[C@@H]',
 '[C@@]',
 '[C@H]',
 '[C@]',
 '[N+]',
 '[N-]',
 '[O-]',
 '[O]',
 '[S+]',
 '[n+]',
 '[nH]',
 '\\',
 'c',
 'n',
 'o',
 's'}

In [13]:
pc_s1 = pd.read_table('../datasets/PubChem/PubChem_sample_1.txt', header=None, usecols=[1])
pc_s2 = pd.read_table('../datasets/PubChem/PubChem_sample_2.txt', header=None, usecols=[1])
pc_s3 = pd.read_table('../datasets/PubChem/PubChem_sample_3.txt', header=None, usecols=[1])
pc_s4 = pd.read_table('../datasets/PubChem/PubChem_sample_4.txt', header=None, usecols=[1])
pc_s5 = pd.read_table('../datasets/PubChem/PubChem_sample_5.txt', header=None, usecols=[1])

In [14]:
pubchem = pc_s1.append(pc_s2)
pubchem = pubchem.append(pc_s3)
pubchem = pubchem.append(pc_s4)
pubchem = pubchem.append(pc_s5)

In [15]:
pubchem.dropna(how='any', inplace=True)
pubchem.reset_index(drop=True)

,1
0,C1CC(=O)N(C1=O)CCC#N
1,CC(CCC(=O)C(C)C1C(=O)CC2[C@]1(CCC3[C@@H]2CC=C4...
2,CC(C)[C@@H](C1=CC=C(C=C1)Cl)C(=O)NC2=NC=CS2
3,COC1=CC=C(C=C1)N(CC(=O)NC2=CC(=CC=C2)SC)S(=O)(...
4,CC1=CC(=CC=C1)N2C(=O)/C(=C\C3=CC=C(C=C3)O[C@H]...
...,...
995,C1C(N(C2=CC=CC(=C2O1)C3=CC(=CC=C3)OC(F)(F)F)C[...
996,CCC=NC1=CC=CC=C1SC2=CC=CC=C2.CC(/C=C(\C(=C/CC=...
997,C/C=C\C=C(\C1CCN(CC1)C=O)/C(=C\C)/C(F)(F)F
998,CC(C)(O)O.C=CC1CCCC2=C(C3=C(C(=C12)C4=CC=CC=C4...


In [16]:
pubchem.rename(columns={1:'smiles'}, inplace=True)

In [18]:
alphabet_pubchem = set()

for row in pubchem.index:
    tokens = atomwise_tokenizer(pubchem.iloc[row,0])
    alphabet_pubchem.update(tokens)

In [19]:
alphabet_pubchem

{'#',
 '(',
 ')',
 '.',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '=',
 'Br',
 'C',
 'Cl',
 'F',
 'I',
 'N',
 'O',
 'P',
 'S',
 '[B]',
 '[Br-]',
 '[C-]',
 '[C@@H]',
 '[C@@]',
 '[C@H]',
 '[C@]',
 '[CH2]',
 '[HH]',
 '[I-]',
 '[Mg+2]',
 '[N+]',
 '[N-]',
 '[NH+]',
 '[NH2+]',
 '[Ni+2]',
 '[O-]',
 '[Sn]',
 '[Te]',
 '\\'}

## Compare the alphabets

In [20]:
alphabet_lense.difference(alphabet_pubchem) # die Elemente, die nicht in PubChem sind

{'-', '[O]', '[S+]', '[n+]', '[nH]', 'c', 'n', 'o', 's'}

In [21]:
alphabet_pubchem.difference(alphabet_lense) # die Elemente, die nicht in Lense sind

{'.',
 '8',
 '9',
 '[B]',
 '[Br-]',
 '[C-]',
 '[CH2]',
 '[HH]',
 '[I-]',
 '[Mg+2]',
 '[NH+]',
 '[NH2+]',
 '[Ni+2]',
 '[Sn]',
 '[Te]'}

In [22]:
#alphabet_lense.intersection(alphabet_pubchem)

{'#',
 '(',
 ')',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '=',
 'Br',
 'C',
 'Cl',
 'F',
 'I',
 'N',
 'O',
 'P',
 'S',
 '[C@@H]',
 '[C@@]',
 '[C@H]',
 '[C@]',
 '[N+]',
 '[N-]',
 '[O-]',
 '\\'}

Strategy: Throw out the things in PubChem, which contain tokens which aren't in Lense.

In [24]:
print(alphabet_lense)

{'[N-]', '[C@@]', '[C@]', ')', '#', '2', '7', 'F', 'o', 'I', 'O', '[O-]', '/', 'P', 'c', 'C', '(', '[O]', 's', 'Cl', '[N+]', '6', '[n+]', '1', '=', '-', '5', 'N', '\\', '[S+]', 'S', '4', '[nH]', '[C@@H]', 'Br', 'n', '3', '[C@H]'}
